In [25]:
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.model_selection import TimeSeriesSplit
import optuna
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler

In [26]:
X = pd.read_csv('dataset/X_train.csv').set_index('DELIVERY_START')
X.index = pd.to_datetime(X.index, utc=True).tz_convert(None)
X.head()

,load_forecast,coal_power_available,gas_power_available,nucelear_power_available,wind_power_forecasts_average,solar_power_forecasts_average,wind_power_forecasts_std,solar_power_forecasts_std,predicted_spot_price
DELIVERY_START,,,,,,,,,
2022-01-01 01:00:00,49439.0,3386.0,11487.0,44118.0,3035.0,0.0,79.248348,0.0,NaN
2022-01-01 02:00:00,46511.0,3386.0,11487.0,44118.0,3143.0,0.0,61.776532,0.0,NaN
2022-01-01 03:00:00,45158.0,3386.0,11487.0,44118.0,3288.0,0.0,44.291112,0.0,NaN
2022-01-01 04:00:00,44779.0,3386.0,11487.0,44118.0,3447.0,0.0,36.127588,0.0,NaN
2022-01-01 05:00:00,45284.0,3386.0,11487.0,44118.0,3679.0,0.0,30.983023,0.0,NaN


In [27]:
y = pd.read_csv('dataset/y_train.csv').set_index('DELIVERY_START')
y.index = pd.to_datetime(y.index, utc=True).tz_convert(None)

# fix outliers
y.loc["2022-04-04 05:00:00"] = (y.loc["2022-04-04 04:00:00"] + y.loc["2022-04-04 07:00:00"]) / 2
y.loc["2022-04-04 06:00:00"] = (y.loc["2022-04-04 04:00:00"] + y.loc["2022-04-04 07:00:00"]) / 2
y = (y > 0).replace({True: 1, False: 0})

/var/folders/z2/6mbyc8hs4l32stll1bfcl0_00000gn/T/ipykernel_33501/249325722.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y = (y > 0).replace({True: 1, False: 0})


In [28]:
def create_time_features(df):
    df = df.copy()
    df['dayofmonth'] = df.index.day
    df['dayofweek'] = df.index.dayofweek
    df['hour'] = df.index.hour
    return df

In [29]:
# initially try using all variables without preprocessing except 'load_forecast' and 'predicted_spot_price' (too many missing points)
X_cols = ['coal_power_available', 'gas_power_available',
       'nucelear_power_available', 'wind_power_forecasts_average',
       'solar_power_forecasts_average', 'wind_power_forecasts_std',
       'solar_power_forecasts_std']

X = X[X_cols].interpolate()
X = create_time_features(X)

cutoff_date = "2023-03-01"
X_train = X.loc[:cutoff_date]
X_test = X.loc[cutoff_date:]
y_train = y.loc[:cutoff_date]
y_test = y.loc[cutoff_date:]

scaler = StandardScaler()
scaler.fit(X_train[X_cols])
X_train.loc[:, X_cols] = scaler.transform(X_train[X_cols])
X_test.loc[:, X_cols] = scaler.transform(X_test[X_cols])

In [30]:
model = lgb.LGBMClassifier()
model.fit(X_train, y_train.values.squeeze())
y_pred = model.predict(X_test)

[LightGBM] [Info] Number of positive: 4864, number of negative: 5071
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000260 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1411
[LightGBM] [Info] Number of data points in the train set: 9935, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.489582 -> initscore=-0.041677
[LightGBM] [Info] Start training from score -0.041677


In [36]:
f1_score(y_test.values.squeeze(), y_pred)

0.625866050808314